In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
import os

In [3]:
current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

grandparent_dir = os.path.abspath(os.path.join(parent_dir, os.pardir))

csv_file_path = os.path.join(grandparent_dir, 'crop_recommendation.csv')

In [4]:
# Load the data
data = pd.read_csv(csv_file_path)

In [5]:
# Separate features and labels
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [6]:
# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
# Number of classes
num_classes = len(label_encoder.classes_)

# Convert labels to one-hot encoded vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [10]:
# Create the ANN model
def create_model(hidden_units=128, dropout_rate=0.2):
    model = keras.Sequential()
    model.add(layers.Dense(hidden_units, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(layers.Dense(hidden_units, activation='relu'))
    model.add(layers.Dense(hidden_units, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
# Define the hyperparameters to tune
param_grid = {
    'hidden_units': [64, 128, 256],
    'dropout_rate': [0.2, 0.4, 0.6]
}

In [12]:
# Create the model
model = keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=10, verbose=0)

C:\Users\User\AppData\Local\Temp\ipykernel_23752\2528482199.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=10, verbose=0)


In [13]:
# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid_search.fit(X_train, y_train)

In [14]:
# Print the best hyperparameters and accuracy
print("Best Hyperparameters: ", grid_result.best_params_)
print("Best Accuracy: {:.2f}".format(grid_result.best_score_))

Best Hyperparameters:  {'dropout_rate': 0.2, 'hidden_units': 256}
Best Accuracy: 0.97


In [16]:
# Evaluate the best model on the test set
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)
loss, accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss*100:.4f}')
print(f'Test accuracy: {accuracy*100:.4f}')

14/14 [==============================] - 0s 3ms/step


AttributeError: 'KerasClassifier' object has no attribute 'evaluate'